In [ ]:
# 重构于notebooks文件夹下

import torch
from torch import nn
from d2l import torch as d2l
from IPython import display

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 初始化模型参数

这个代码创建了一个简单的神经网络模型，并初始化了其权重。具体来说，这段代码的功能如下：

1. **创建网络结构**:
   ```python
   net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))
   ```
   - `nn.Sequential`：一个顺序容器，模块将按传入的顺序添加到计算图中。
   - `nn.Flatten`：将输入展平为一维张量。假设输入是 `28x28` 的图像，它将变成 `784` 的一维向量。
   - `nn.Linear(784, 10)`：一个全连接层，输入维度为 `784`，输出维度为 `10`，通常用于分类任务中的 `10` 个类别。

2. **初始化权重函数**:
   ```python
   def init_weights(m):
       if type(m) == nn.Linear:
           nn.init.normal_(m.weight, std=0.01)
   ```
   - `init_weights`：一个函数，用于初始化线性层的权重。
   - `nn.init.normal_`：用均值为 `0`，标准差为 `0.01` 的正态分布初始化权重。

3. **应用初始化权重函数**:
   ```python
   net.apply(init_weights)
   ```
   - `apply`：一个递归的函数，应用于 `net` 中的所有模块。如果模块是 `nn.Linear`，则使用 `init_weights` 函数来初始化其权重。

总结来说，这段代码定义了一个简单的全连接神经网络，并将其权重初始化为均值为 `0`，标准差为 `0.01` 的正态分布。

In [ ]:
# PyTorch不会隐式地调整输入的形状。因此，
# 我们在线性层前定义了展平层（flatten），来调整网络输入的形状
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

# 重新审视Softmax的实现

In [ ]:
loss = nn.CrossEntropyLoss(reduction='none')

这行代码定义了一个交叉熵损失函数，并指定不对每个样本的损失进行求和或求平均，即保留每个样本的损失值。让我们更详细地解释一下：

```python
import torch.nn as nn

loss = nn.CrossEntropyLoss(reduction='none')
```

### 详细解释

1. **`nn.CrossEntropyLoss`**:
    - `nn.CrossEntropyLoss` 是 PyTorch 中用于多分类问题的损失函数。
    - 它结合了 `nn.LogSoftmax` 和 `nn.NLLLoss`，对输入应用 LogSoftmax 函数，并计算负对数似然。

2. **参数 `reduction`**:
    - `reduction` 参数控制对输出的处理方式。
    - `reduction='none'`：不对损失值进行任何归约操作，即保留每个样本的损失值。
    - 其他选项：
        - `reduction='mean'`：对所有样本的损失值求平均。
        - `reduction='sum'`：对所有样本的损失值求和。

### 使用场景

使用 `reduction='none'` 的主要场景包括：

- 需要对每个样本的损失值进行进一步处理或分析。
- 在某些自定义的训练逻辑中，需要对每个样本的损失值进行不同的加权或筛选。

例如，您可能需要在自定义的训练过程中对某些特定样本施加更大的权重，或者在不同样本上应用不同的损失函数。通过保留每个样本的损失值，可以灵活地实现这些需求。

# 优化算法

In [ ]:
# 在这里，我们使用学习率为0.1的小批量随机梯度下降作为优化算法。 这与我们在线性回归例子中的相同，这说明了优化器的普适性。
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

# 训练

In [ ]:
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    # 它首先将预测的概率分布转化为类别标签（如果需要的话），然后比较这些预测标签与真实标签，最终计算预测正确的数量。
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

class Animator:  #@save
    """在动画中绘制数据"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # 向图表中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)
        
def train_epoch_ch3(net, train_iter, loss, updater):  #@save
    """训练模型一个迭代周期（定义见第3章）"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]

def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):  #@save
    """训练模型（定义见第3章）"""
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [ ]:
num_epochs = 10
train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)